## Profile(s)

**profile**: A series of connected observations along a vertical line. Each profile has only one lat, lon coordinate (possibly nominal), so that the points along the profile differ only in z coordinate and possibly time coodinate. There can be multiple profiles in the same file, and each profile has a unique identifier. If we have many profiles with the same lat, lon location, use the Time Series Profile type. *Examples: atomospheric profiles from satellites, moving profilers*.

* [Only one profile in the file](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_single_profile)
* [All profiles have the same vertical coordinates](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_orthogonal_multidimensional_array_representation_of_profiles)
* [Each profile has the same number of vertical coordinates but the coordinate values may be different](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_incomplete_multidimensional_array_representation_of_profiles)
* Each profile has a different number of vertical coordinates and we wanna keep file size as small as possible:
    * [we have all the data already, and we wanna optimize reading all the data for one profile](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_contiguous_ragged_array_representation_of_profiles)
    * [we wanna write the data as it arrives, in any order](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_indexed_ragged_array_representation_of_profiles)
        * *double time(profile) is possible*; *double time(obs)* is also possible, when the observation varies by time. 


Example Dataset: [CLAPPP : New Caledonian lagoons: CTD Profiles](https://erddap.osupytheas.fr/erddap/files/CLAPP_CTD_OutNetCDF_6c83/Clappp1/)

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
os.chdir('/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data')
os.getcwd()

'/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data'

In [3]:
pf_files = glob(os.path.join(os.getcwd(), "dsg_profile", "*.nc"))
pf_files

['/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_profile/prony-1.nc',
 '/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_profile/teremba-1.nc']

### [H.3.3.](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_single_profile) Single Profile

In [4]:
# Load the profile dataset.
# It is a standard single profile netCDF file.
ds_prony = xr.open_dataset(pf_files[0], decode_times=False)
ds_prony

<xarray.Dataset> Size: 382B
Dimensions:       (depth: 11)
Coordinates:
    time          int32 4B ...
    latitude      float32 4B ...
    longitude     float32 4B ...
  * depth         (depth) float32 44B -0.1 -0.2 -0.3 -0.4 ... -0.9 -1.0 -1.1
Data variables:
    stationname   |S18 18B ...
    temperature   (depth) float32 44B ...
    conductivity  (depth) float32 44B ...
    salinity      (depth) float32 44B ...
    fluorescence  (depth) float32 44B ...
    irradiance    (depth) float32 44B ...
    density       (depth) float32 44B ...
    turbidity     (depth) float32 44B ...
Attributes: (12/37)
    description:              CTD profile (NetCDF files) for station ./Clappp...
    title:                    CLAPPP Project  : CTD profile (NetCDF files) st...
    keywords:                 Seabird CTD SBE19plus, temperature, conductivit...
    history:                  Created 02/06/21
    production:               MIO UMR 7294 CNRS / OSU Pytheas UMS3470 CNRS
    contact:                  martine Rodier (martine.rodier@mio.osupytheas.fr)
    ...                       ...
    contributor_institution:  MIO UMR 7294 CNRS / OSU Pytheas UMS 3470 CNRS
    institution:              MIO UMR7294 CNRS / OSU Pytheas
    publisher_name:           MIO Marine Institute of Oceanography - OSU Pytheas
    publisher_email:          martine.rodier@mio.osupytheas.fr
    publisher_url:            https://dataset.osupytheas.fr/
    publisher_institution:    MIO UMR 7294 CNRS / OSU Pytheas UMS 3470 CNRS

### [H.3.1.](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_orthogonal_multidimensional_array_representation_of_profiles) Orthogonal multidimensional array representation of profiles

Multiple profiles, same number of vertical levels and vertical coordinate values are identical.

In [5]:
# Load another profile.
ds_teremba = xr.open_dataset(pf_files[1], decode_times=False)

In [6]:
# Vertical levels of two datasets
print("Depth Coordinates at Station 'prony': ", ds_prony.depth.data)
print("Depth Coordinates at Station 'teremba': ", ds_teremba.depth.data[:20])

# Select the same depths for the station "teremba" to match those for the station "prony"
ds_teremba = ds_teremba.sel(depth=ds_prony.depth.data)
print("New Depth Coordinates at Station 'teremba': ", ds_teremba.depth.data)

Depth Coordinates at Station 'prony':  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1]
Depth Coordinates at Station 'teremba':  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1 -1.2 -1.3 -1.4
 -1.5 -1.6 -1.7 -1.8 -1.9 -2. ]
New Depth Coordinates at Station 'teremba':  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1]


In [7]:
# Vertical coordinate values for both profiles
depth = ds_teremba.depth.data
print("Vertical levels for both profiles are: ", depth)

# Station names for two profiles
# class <bytes>.decode() ==> string
station_name = [ds_prony.stationname.data.tolist().decode(), ds_teremba.stationname.data.tolist().decode()]
print("Station names of two profiles are: ", station_name)

# Time coordinate values for two profiles
# 0-D array (scalar).tolist() ==> the scalar value itself, not a list containing that value
time = [ds_prony.time.data.tolist(), ds_teremba.time.data.tolist()]
print("Time values for two profiles are: ", time)

# Longitudes for two profiles
lon = [ds_prony.longitude.data.tolist(), ds_teremba.longitude.data.tolist()]
print("Longitudes for two stations are: ", lon)

# Latitudes for two profiles
lat = [ds_prony.latitude.data.tolist(), ds_teremba.latitude.data.tolist()]
print("Latitudes for two profiles are: ", lat)

Vertical levels for both profiles are:  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1]
Station names of two profiles are:  ['./Clappp1//prony-1', './Clappp1//teremba-1']
Time values for two profiles are:  [20439933, 20442692]
Longitudes for two stations are:  [166.8699951171875, 165.72999572753906]
Latitudes for two profiles are:  [-22.399999618530273, -21.850000381469727]


In [8]:
# In the example, we have 2 profiles and 11 vertical coordinates,
# Make all the data variables be an array of the shape (profile, z), i.e. (2, 11)
temperature = np.row_stack((ds_prony.temperature.data.tolist(), ds_teremba.temperature.data.tolist()))
conductivity = np.row_stack((ds_prony.conductivity.data.tolist(), ds_teremba.conductivity.data.tolist()))
salinity = np.row_stack((ds_prony.salinity.data.tolist(), ds_teremba.salinity.data.tolist()))
fluorescence = np.row_stack((ds_prony.fluorescence.data.tolist(), ds_teremba.fluorescence.data.tolist()))
irradiance = np.row_stack((ds_prony.irradiance.data.tolist(), ds_teremba.irradiance.data.tolist()))
density = np.row_stack((ds_prony.density.data.tolist(), ds_teremba.density.data.tolist()))
turbidity = np.row_stack((ds_prony.turbidity.data.tolist(), ds_teremba.turbidity.data.tolist()))

In [9]:
# Get Attributes
attrs_depth = ds_prony.depth.attrs
attrs_station_name = ds_prony.stationname.attrs
attrs_lat = ds_prony.latitude.attrs
attrs_lon = ds_prony.longitude.attrs
attrs_time = ds_prony.time.attrs

attrs_temperature = ds_prony.temperature.attrs
attrs_conductivity = ds_prony.conductivity.attrs
attrs_salinity = ds_prony.salinity.attrs
attrs_fluorescence = ds_prony.fluorescence.attrs
attrs_irradiance = ds_prony.irradiance.attrs
attrs_density = ds_prony.density.attrs
attrs_turbidity = ds_prony.turbidity.attrs

attrs_global = ds_prony.attrs

In [10]:
ds_profile = xr.Dataset(
    coords={
        "time":(["station"], time, attrs_time),
        "lon":(["station"], lon, attrs_lon),
        "lat":(["station"], lat, attrs_lat),
        "station_name":(["station"], station_name, attrs_station_name),
        "depth":(["depth"], depth, attrs_depth)
    },
    data_vars={
        "temperature":(["station","depth"], temperature, attrs_temperature),
        "conductivity":(["station","depth"], conductivity, attrs_conductivity),
        "salinity":(["station","depth"], salinity, attrs_salinity),
        "fluorescence":(["station","depth"], fluorescence, attrs_fluorescence),
        "irradiance":(["station","depth"], irradiance, attrs_irradiance),
        "density":(["station","depth"], density, attrs_density),
        "turbidity":(["station","depth"], turbidity, attrs_turbidity)
    },
    attrs={
        "featureType":"profile"
    }
)

# Add attribute "cf_role"
ds_profile.station_name.attrs['cf_role'] = "profile_id"

ds_profile

<xarray.Dataset> Size: 1kB
Dimensions:       (station: 2, depth: 11)
Coordinates:
    time          (station) int64 16B 20439933 20442692
    lon           (station) float64 16B 166.9 165.7
    lat           (station) float64 16B -22.4 -21.85
    station_name  (station) <U20 160B './Clappp1//prony-1' './Clappp1//teremb...
  * depth         (depth) float32 44B -0.1 -0.2 -0.3 -0.4 ... -0.9 -1.0 -1.1
Dimensions without coordinates: station
Data variables:
    temperature   (station, depth) float64 176B 24.47 24.32 ... 24.67 24.67
    conductivity  (station, depth) float64 176B 32.59 52.66 ... 53.09 53.08
    salinity      (station, depth) float64 176B 20.58 35.22 ... 35.27 35.27
    fluorescence  (station, depth) float64 176B 1.813 1.776 ... 0.3914 0.3622
    irradiance    (station, depth) float64 176B -999.0 -999.0 ... -999.0 -999.0
    density       (station, depth) float64 176B 1.013e+03 ... 1.024e+03
    turbidity     (station, depth) float64 176B 0.0 0.0 0.0 ... 0.148 0.0 0.0
Attributes:
    featureType:  profile

#### Alternative Approach: Loop

When there're lots of data variables in a dataset, it can get tedious to access to the arrays one by one; Looping is an alternative way of doing.

In [11]:
# Alternative way to access to all arrays and reshape them.
coord_names = list(ds_prony.coords)[:-1] # drop "depth" from the list
data_variable_names = list(ds_prony.keys())[1:] # drop "stationname" from the list

print(coord_names)
print(data_variable_names)

['time', 'latitude', 'longitude']
['temperature', 'conductivity', 'salinity', 'fluorescence', 'irradiance', 'density', 'turbidity']


In [12]:
# Create a dictionary containing the coordinates for the new dataset
dict_coords = {}
for i in coord_names:
    dict_coords[i] = [ds_prony[i].data.tolist(), ds_teremba[i].data.tolist()]

dict_coords

{'time': [20439933, 20442692],
 'latitude': [-22.399999618530273, -21.850000381469727],
 'longitude': [166.8699951171875, 165.72999572753906]}

In [13]:
# Create another dictionary containing the data variables for the new dataset
dict_datavar = {}
for i in data_variable_names:
    dict_datavar[i] = np.row_stack((ds_prony[i].data.tolist(), ds_teremba[i].data.tolist()))

dict_datavar

{'temperature': array([[24.46899986, 24.31800079, 24.35400009, 24.34300041, 24.27099991,
         24.25600052, 24.24399948, 24.23600006, 24.22999954, 24.23999977,
         24.3239994 ],
        [25.02899933, 24.91600037, 24.84300041, 24.7840004 , 24.76199913,
         24.75099945, 24.70299911, 24.69300079, 24.67700005, 24.66900063,
         24.66500092]]),
 'conductivity': array([[32.59090042, 52.65810013, 52.72539902, 52.71509933, 52.65539932,
         52.64300156, 52.64400101, 52.64849854, 52.71120071, 52.72040176,
         52.76760101],
        [21.35740089, 34.02090073, 25.94919968, 40.42350006, 47.11520004,
         46.70410156, 53.00569916, 53.01900101, 53.10960007, 53.08599854,
         53.08349991]]),
 'salinity': array([[20.58300018, 35.22499847, 35.24599838, 35.2480011 , 35.25899887,
         35.26200104, 35.27199936, 35.28099823, 35.33399963, 35.33300018,
         35.30199814],
        [12.79100037, 21.37000084, 15.89099979, 25.94199944, 30.78700066,
         30.49399948, 35

In [14]:
# Get Attributes
attrs_coords = {}
for i in coord_names:
    attrs_coords[i] = ds_prony[i].attrs

attrs_dataVar = {}
for i in data_variable_names:
    attrs_dataVar[i] = ds_prony[i].attrs

attrs_depth = ds_prony.depth.attrs
attrs_station_name = ds_prony.stationname.attrs

attrs_global = ds_prony.attrs

In [15]:
# Make a dictionary containing the data variables
dict_data_vars = {}
for i in data_variable_names:
    dict_data_vars[i] = (["station","depth"], dict_datavar[i], attrs_dataVar[i])

# Compose the xarray dataset
ds_profile = xr.Dataset(
    coords={
        "depth": (["depth"], np.float32(depth), attrs_depth),
        "station": (["station"], station_name, attrs_station_name),
        "time": (["station"], np.int32(dict_coords['time']), attrs_coords['time']),
        "lat": (["station"], np.float32(dict_coords['latitude']), attrs_coords['latitude']),
        "lon": (["station"], np.float32(dict_coords['longitude']), attrs_coords['longitude'])
    },
    data_vars = dict_data_vars,
)

In [16]:
# Add Global Attributes
ds_profile.attrs = attrs_global

# Add Feature Type 
ds_profile.attrs["featureType"] = "profile"

In [17]:
ds_profile

<xarray.Dataset> Size: 1kB
Dimensions:       (station: 2, depth: 11)
Coordinates:
  * depth         (depth) float32 44B -0.1 -0.2 -0.3 -0.4 ... -0.9 -1.0 -1.1
  * station       (station) <U20 160B './Clappp1//prony-1' './Clappp1//teremb...
    time          (station) int32 8B 20439933 20442692
    lat           (station) float32 8B -22.4 -21.85
    lon           (station) float32 8B 166.9 165.7
Data variables:
    temperature   (station, depth) float64 176B 24.47 24.32 ... 24.67 24.67
    conductivity  (station, depth) float64 176B 32.59 52.66 ... 53.09 53.08
    salinity      (station, depth) float64 176B 20.58 35.22 ... 35.27 35.27
    fluorescence  (station, depth) float64 176B 1.813 1.776 ... 0.3914 0.3622
    irradiance    (station, depth) float64 176B -999.0 -999.0 ... -999.0 -999.0
    density       (station, depth) float64 176B 1.013e+03 ... 1.024e+03
    turbidity     (station, depth) float64 176B 0.0 0.0 0.0 ... 0.148 0.0 0.0
Attributes: (12/37)
    description:              CTD profile (NetCDF files) for station ./Clappp...
    title:                    CLAPPP Project  : CTD profile (NetCDF files) st...
    keywords:                 Seabird CTD SBE19plus, temperature, conductivit...
    history:                  Created 02/06/21
    production:               MIO UMR 7294 CNRS / OSU Pytheas UMS3470 CNRS
    contact:                  martine Rodier (martine.rodier@mio.osupytheas.fr)
    ...                       ...
    contributor_institution:  MIO UMR 7294 CNRS / OSU Pytheas UMS 3470 CNRS
    institution:              MIO UMR7294 CNRS / OSU Pytheas
    publisher_name:           MIO Marine Institute of Oceanography - OSU Pytheas
    publisher_email:          martine.rodier@mio.osupytheas.fr
    publisher_url:            https://dataset.osupytheas.fr/
    publisher_institution:    MIO UMR 7294 CNRS / OSU Pytheas UMS 3470 CNRS

### [H.3.2.](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_incomplete_multidimensional_array_representation_of_profiles) Incomplete multidimensional array representation of profiles

Recommended for the situation where there are multiple profiles and same number of vertical coordinates in each, but coordinate values are different.

In [29]:
# Load another profile.
ds_prony = xr.open_dataset(pf_files[0], decode_times=False)
ds_teremba = xr.open_dataset(pf_files[1], decode_times=False)

# Vertical levels of two datasets
print("Depth Coordinates at Station 'prony': ", ds_prony.depth.data)
print("Depth Coordinates at Station 'teremba': ", ds_teremba.depth.data[0:21])

# Select the same depths for the station "teremba" to match those for the station "prony"
ds_teremba = ds_teremba.isel(depth=slice(10,21))
print("New Depth Coordinates at Station 'teremba': ", ds_teremba.depth.data)

Depth Coordinates at Station 'prony':  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1]
Depth Coordinates at Station 'teremba':  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1 -1.2 -1.3 -1.4
 -1.5 -1.6 -1.7 -1.8 -1.9 -2.  -2.1]
New Depth Coordinates at Station 'teremba':  [-1.1 -1.2 -1.3 -1.4 -1.5 -1.6 -1.7 -1.8 -1.9 -2.  -2.1]


In [49]:
# Compared to the previous example in H.3.1, 
# replace the coordinate variable depth(depth) with depth(station,depth).

# Station names for two profiles
# class <bytes>.decode() ==> string
station_name = [ds_prony.stationname.data.tolist().decode(), ds_teremba.stationname.data.tolist().decode()]
print("Station names of two profiles are: ", station_name)
# Get attributes of station names
attrs_station_name = ds_prony.stationname.attrs

# Depths at two stations
depths = np.row_stack((ds_prony.depth.data.tolist(), ds_teremba.depth.data.tolist()))
print("Depths at two stations are: ", depths)
# Get attributes of depth
attrs_depth = ds_prony.depth.attrs

Station names of two profiles are:  ['./Clappp1//prony-1', './Clappp1//teremba-1']
Depths at two stations are:  [[-0.1        -0.2        -0.30000001 -0.40000001 -0.5        -0.60000002
  -0.69999999 -0.80000001 -0.89999998 -1.         -1.10000002]
 [-1.10000002 -1.20000005 -1.29999995 -1.39999998 -1.5        -1.60000002
  -1.70000005 -1.79999995 -1.89999998 -2.         -2.0999999 ]]


In [50]:
# Get list of auxiliary coordinate variable names
ls_auxCoords = list(ds_prony.coords)[:-1] # Drop Depth
print("List of auxiliary coordinate variables: ", ls_auxCoords)

# Get time, lon, lat and depth by looping
auxCoords = {}
for i in ls_auxCoords:
    auxCoords[i] = [ds_prony[i].data.tolist(), ds_teremba[i].data.tolist()]

    # Alternatively: If there are lots of datasets, we can consider of using nested loops.
    #auxCoords[i] = [j[i].data.tolist() for j in [ds_prony, ds_teremba]]
print(auxCoords)

# Get Attributes of auxiliary coordinate variables
auxCoords_attrs = {}
for i in ls_auxCoords:
    auxCoords_attrs[i] = ds_prony[i].attrs

List of auxiliary coordinate variables:  ['time', 'latitude', 'longitude']
{'time': [20439933, 20442692], 'latitude': [-22.399999618530273, -21.850000381469727], 'longitude': [166.8699951171875, 165.72999572753906]}


In [51]:
# Get list of data variables
ls_dataVar = list(ds_prony.keys())[1:] # Drop station names from the list
print("List of data variables: ", ls_dataVar)

# Get data variables by looping
dataVar = {}
for i in ls_dataVar:
    dataVar[i] = np.row_stack((ds_prony[i].data.tolist(), ds_teremba[i].data.tolist()))

# Get Attributes of data variables
dataVar_attrs = {}
for i in ls_dataVar:
    dataVar_attrs[i] = ds_prony[i].attrs


List of data variables:  ['temperature', 'conductivity', 'salinity', 'fluorescence', 'irradiance', 'density', 'turbidity']


In [57]:
# Create new Dataset
dict_auxCoords = {}
for i in ls_auxCoords:
    dict_auxCoords[i] = (["station"], auxCoords[i], auxCoords_attrs[i])
# Add station name and depth
dict_auxCoords["station_name"] = (["station"], station_name, attrs_station_name)
dict_auxCoords["depths"] = (["station","depth"], depths, attrs_depth)

dict_dataVar = {}
for i in ls_dataVar:
    dict_dataVar[i] = (["station","depth"], dataVar[i], dataVar_attrs[i])

ds_profile = xr.Dataset(coords=dict_auxCoords, 
                        data_vars=dict_dataVar,
                        attrs={"featureType":"profile"})

# Add attribute "cf_role"
ds_profile.station_name.attrs['cf_role'] = "profile_id"

ds_profile

<xarray.Dataset> Size: 2kB
Dimensions:       (station: 2, depth: 11)
Coordinates:
    time          (station) int64 16B 20439933 20442692
    latitude      (station) float64 16B -22.4 -21.85
    longitude     (station) float64 16B 166.9 165.7
    station_name  (station) <U20 160B './Clappp1//prony-1' './Clappp1//teremb...
    depths        (station, depth) float64 176B -0.1 -0.2 -0.3 ... -2.0 -2.1
Dimensions without coordinates: station, depth
Data variables:
    temperature   (station, depth) float64 176B 24.47 24.32 ... 24.65 24.65
    conductivity  (station, depth) float64 176B 32.59 52.66 ... 53.07 53.06
    salinity      (station, depth) float64 176B 20.58 35.22 ... 35.27 35.27
    fluorescence  (station, depth) float64 176B 1.813 1.776 ... 0.2968 0.3272
    irradiance    (station, depth) float64 176B -999.0 -999.0 ... -999.0 -999.0
    density       (station, depth) float64 176B 1.013e+03 ... 1.024e+03
    turbidity     (station, depth) float64 176B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    featureType:  profile

### [H.3.4.](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_contiguous_ragged_array_representation_of_profiles) Contiguous ragged array of representation of profiles

Recommeded for the situation where there are multiple profiles, each profile has different number of vertical coordinates, and the dataset is complete.

In [58]:
# Load another profile.
ds_prony = xr.open_dataset(pf_files[0], decode_times=False)
ds_teremba = xr.open_dataset(pf_files[1], decode_times=False)

print("Number of observatinos at station 'prony': ", len(ds_prony.depth))
print("Number of observations at station 'teremba': ", len(ds_teremba.depth))

Number of observatinos at station 'prony':  11
Number of observations at station 'teremba':  604


In [63]:
# Number of observations at each station
rowSize = [len(ds_prony.depth), len(ds_teremba.depth)]
print("Number of observations at each station is: ", rowSize)

# Station names for two profiles
# class <bytes>.decode() ==> string
station_name = [ds_prony.stationname.data.tolist().decode(), ds_teremba.stationname.data.tolist().decode()]
print("Station names of two profiles are: ", station_name)

# Time coordinate values for two profiles
# 0-D array (scalar).tolist() ==> the scalar value itself, not a list containing that value
time = [ds_prony.time.data.tolist(), ds_teremba.time.data.tolist()]
print("Time values for two profiles are: ", time)

# Longitudes for two profiles
lon = [ds_prony.longitude.data.tolist(), ds_teremba.longitude.data.tolist()]
print("Longitudes for two stations are: ", lon)

# Latitudes for two profiles
lat = [ds_prony.latitude.data.tolist(), ds_teremba.latitude.data.tolist()]
print("Latitudes for two profiles are: ", lat)

# Vertical coordinate values for both profiles
depth = np.concatenate([ds_prony.depth.data, ds_teremba.depth.data])
print("Depths of the (first twenty) individual observations are: ", depth[:20])

Number of observations at each station is:  [11, 604]
Depths of the (first twenty) individual observations are:  [-0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.  -1.1 -0.1 -0.2 -0.3
 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9]
Station names of two profiles are:  ['./Clappp1//prony-1', './Clappp1//teremba-1']
Time values for two profiles are:  [20439933, 20442692]
Longitudes for two stations are:  [166.8699951171875, 165.72999572753906]
Latitudes for two profiles are:  [-22.399999618530273, -21.850000381469727]


In [71]:
# Merge each data variable of two stations to one-dimensional array
temperature = np.concatenate([ds_prony.temperature.data, ds_teremba.temperature.data])
conductivity = np.concatenate([ds_prony.conductivity.data, ds_teremba.conductivity.data])
salinity = np.concatenate([ds_prony.salinity.data, ds_teremba.salinity.data])
fluorescence = np.concatenate([ds_prony.fluorescence.data, ds_teremba.fluorescence.data])
irradiance = np.concatenate([ds_prony.irradiance.data, ds_teremba.irradiance.data])
density = np.concatenate([ds_prony.density.data, ds_teremba.density.data])
turbidity = np.concatenate([ds_prony.turbidity.data, ds_teremba.turbidity.data])

In [96]:
# Get Attributes
attrs_depth = ds_prony.depth.attrs
attrs_station_name = ds_prony.stationname.attrs
attrs_lat = ds_prony.latitude.attrs
attrs_lon = ds_prony.longitude.attrs
attrs_time = ds_prony.time.attrs

attrs_temperature = ds_prony.temperature.attrs
attrs_conductivity = ds_prony.conductivity.attrs
attrs_salinity = ds_prony.salinity.attrs
attrs_fluorescence = ds_prony.fluorescence.attrs
attrs_irradiance = ds_prony.irradiance.attrs
attrs_density = ds_prony.density.attrs
attrs_turbidity = ds_prony.turbidity.attrs

attrs_global = ds_prony.attrs

In [97]:
# Create new Dataset
ds_profile = xr.Dataset(
    coords={
        "station_name": (["station"], station_name, attrs_station_name),
        "time": (["station"], time, attrs_time),
        "lon": (["station"], lon, attrs_lon),
        "lat": (["station"], lat, attrs_lat),
        "rowSize": (["station"], rowSize, {"long_name": "number of observations for this profile",
                                           "sample_dimension": "obs"}),
        "depth": (["obs"], depth, attrs_depth)
    },
    data_vars={
        "temperature":(["obs"], temperature, attrs_temperature),
        "conductivity":(["obs"], conductivity, attrs_conductivity),
        "salinity":(["obs"], salinity, attrs_salinity),
        "fluorescence":(["obs"], fluorescence, attrs_fluorescence),
        "irradiance":(["obs"], irradiance, attrs_irradiance),
        "density":(["obs"], density, attrs_density),
        "turbidity":(["obs"], turbidity, attrs_turbidity)
    },
    attrs={
        "featureType": "profile"
    }
)

# Add attribute "cf_role"
ds_profile.station_name.attrs['cf_role'] = "profile_id"

ds_profile

<xarray.Dataset> Size: 20kB
Dimensions:       (obs: 615, station: 2)
Coordinates:
    station_name  (station) <U20 160B './Clappp1//prony-1' './Clappp1//teremb...
    time          (station) int64 16B 20439933 20442692
    lon           (station) float64 16B 166.9 165.7
    lat           (station) float64 16B -22.4 -21.85
    rowSize       (station) int64 16B 11 604
    depth         (obs) float32 2kB -0.1 -0.2 -0.3 -0.4 ... -72.2 -72.3 -72.4
Dimensions without coordinates: obs, station
Data variables:
    temperature   (obs) float32 2kB 24.47 24.32 24.35 ... 22.99 22.99 22.99
    conductivity  (obs) float32 2kB 32.59 52.66 52.73 ... 51.46 51.46 51.46
    salinity      (obs) float32 2kB 20.58 35.22 35.25 ... 35.35 35.35 35.35
    fluorescence  (obs) float32 2kB 1.813 1.776 1.756 ... 0.8875 0.8629 0.8641
    irradiance    (obs) float32 2kB -999.0 -999.0 -999.0 ... -999.0 -999.0
    density       (obs) float32 2kB 1.013e+03 1.024e+03 ... 1.025e+03 1.025e+03
    turbidity     (obs) float32 2kB 0.0 0.0 0.0 0.0 ... 0.095 0.084 0.1 0.091
Attributes:
    featureType:  profile

### [H.3.5.](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_indexed_ragged_array_representation_of_profiles) Indexed ragged array representation of profiles

Recommended for the situation where there are multiple profiles, each profile has differrent number of vertical coordinates, and the dataset is not completed, i.e. there is the need to add more data to the existing NetCDF file.

In [76]:
# Load another profile.
ds_prony = xr.open_dataset(pf_files[0], decode_times=False)
ds_teremba = xr.open_dataset(pf_files[1], decode_times=False)

print("Number of observatinos at station 'prony': ", len(ds_prony.depth))
print("Number of observations at station 'teremba': ", len(ds_teremba.depth))

Number of observatinos at station 'prony':  11
Number of observations at station 'teremba':  604


In [ ]:


# Number of observations at each station
rowSize = [len(ds_prony.depth), len(ds_teremba.depth)]
print("Number of observations at each station is: ", rowSize)

# Station names for two profiles
# class <bytes>.decode() ==> string
station_name = [ds_prony.stationname.data.tolist().decode(), ds_teremba.stationname.data.tolist().decode()]
print("Station names of two profiles are: ", station_name)

# Time coordinate values for two profiles
# 0-D array (scalar).tolist() ==> the scalar value itself, not a list containing that value
time = [ds_prony.time.data.tolist(), ds_teremba.time.data.tolist()]
print("Time values for two profiles are: ", time)

# Longitudes for two profiles
lon = [ds_prony.longitude.data.tolist(), ds_teremba.longitude.data.tolist()]
print("Longitudes for two stations are: ", lon)

# Latitudes for two profiles
lat = [ds_prony.latitude.data.tolist(), ds_teremba.latitude.data.tolist()]
print("Latitudes for two profiles are: ", lat)

In [ ]:
# Vertical coordinate values for both profiles
depth = np.concatenate([ds_prony.depth.data, ds_teremba.depth.data])
print("Depths of the (first twenty) individual observations are: ", depth[:20])
